# Data Cleaning

ps: *Don't forget Garbage In, Garbage out.*

## Objectives:

Our objectives for this notebook are as following:

- Prepare the data sets for further analysis.
- Load and inspect the data prepared during data collection.
- Correlation and PPS study.
- Data Cleaning.
- Conclusion and next steps.


## Inputs:

- inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv
- inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/refurbished_houses.csv


## Outputs:

- outputs/datasets/cleaned/train_set.csv
- outputs/datasets/cleaned/test_set.csv
- outputs/datasets/cleaned/clean_house_price_records.csv
- outputs/datasets/cleaned/clean_refurbished_houses.csv

### Additional Comments:

- As previously mentioned this projects relays on the guidelines provided in the walk through projects, and the lessons provided along the course (especially in the Predictive Analysis)
- Therefore, this notebook will be relating the Data Preparation step of Crisp-DM methodology. 

___


## Change working directory.

- Change the working directory from its current folder to its parent folder: 
    - Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory:

- os.path dirname() gets the parent directory.
- os.chir() defines the new current directory.

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory, well done!")

- Confirm the new current directory:

In [ ]:
current_dir = os.getcwd()
current_dir

___

## Import the necessary packages and set environmental variables:

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
from pandas_profiling import ProfileReport
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from sklearn.pipeline import Pipeline

- Load the data downloaded in the data collection notebook:

In [ ]:
df = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv")
print(df.shape)

In [ ]:
df_refurbished = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/refurbished_houses.csv")
print(df_refurbished.shape)
df_refurbished

____

## Data Exploration

- We need to explore the dataset, check variable types and distributing, missing levels and what value these variables my add in the content of the first business requirement. 

- First we will need to list the variables that are missing a value: 

In [ ]:
vars_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_missing_data

- We need to run the pandas profiling report using only the var_missing_data variable:

In [ ]:
if vars_missing_data:
   pandas_report = ProfileReport(df=df[vars_missing_data], minimal=True)
   pandas_report.to_notebook_iframe()
else:
   print("Done, conclusion? There are no variables that are missing data.")

____

## Correlation and PPS Analysis

- In this section we would like to understand how the target variable, SalePrice, correlates with the features.
- The below code will be from PPS lesson, to help me build the heat maps for pearson and spearman correlation, as well as a PPS heatmap. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

%matplotlib inline

def heatmap_corr(df, threshold, figsize=(20,12), font_annot = 8):
  if len(df.columns) > 1:
    mask = np.zeros_like(df, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    mask[abs(df) < threshold] = True

    fig, axes = plt.subplots(figsize=figsize)
    sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                linewidth=0.5
                     )
    axes.set_yticklabels(df.columns, rotation = 0)
    plt.ylim(len(df.columns),0)
    plt.show()


def heatmap_pps(df, threshold, figsize=(20,12), font_annot = 8):
    if len(df.columns) > 1:

      mask = np.zeros_like(df, dtype=bool)
      mask[abs(df) < threshold] = True

      fig, ax = plt.subplots(figsize=figsize)
      ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                       mask=mask,cmap='rocket_r', annot_kws={"size": font_annot},
                       linewidth=0.05, linecolor='grey')
      
      plt.ylim(len(df.columns),0)
      plt.show()



def CalculateCorrAndPPS(df):
  df_corr_spearman = df.corr(method="spearman")
  df_corr_pearson = df.corr(method="pearson")

  pps_matrix_raw = pps.matrix(df)
  pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

  pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
  print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
  print(pps_score_stats.round(3))

  return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20,12), font_annot=8 ):

  print("\n")
  print("* Here I can analyze how the target variable for your ML models are correlated with other variables (features and target)")
  print("* Analyze multi colinearity, that is, how the features are correlated among themselves")

  print("\n")
  print("*** Heatmap: Spearman Correlation ***")
  print("It evaluates monotonic relationships between variables \n")
  heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Pearson Correlation ***")
  print("It evaluates the linear relationship between two continuous variables \n")
  heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Power Predictive Score (PPS) ***")
  print(f"PPS detects linear or non-linear relationships between two columns.\n"
        f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
  heatmap_pps(df=pps_matrix,threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

- Calculate the correlations and power predictive score:

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

- The table above shows the most common levels for pps scores in the matrix. The majority are between 0 and 066.


Further more we will display the correlation and pps results on the Heat Maps: 

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.6, PPS_Threshold = 0.2,
                  figsize=(12,10), font_annot=10)

___

## Dataset Analysis

### Data Exploration:

- The data presented in these reports shows that there are fields that contain many zero values, more concerning though, is the number of variables that do not contain data. ie. contain null values. 

    - I will further examine these variables and explore whether there is a common criteria that may assist in imputing data into these variables or whether in some case it is viable to drop the feature completely. 
    - I will then do a correlation study and compare the before and after results to establish whether this excise makes a difference to predicting sale price. 


### Correlation and PPS Analysis.

- We should have in mind that the results show a number of variables to be moderated to strong predictors for other variables, most 
asynchronously. However, I am highly interested in variables that are predictors of the sale price.

    - From the results of both the correlation and PPS studies, I see that the strongest predictor of sale price (SalePrice) is Overall Quality (OverallQual) of the property. 
    - Overall the correlation study shows 6 features that are positively and strongly correlated to SalePrice:
        - 1stFlorSF (first floor square foot)
        - GarageArea (garage area measured in square foot)
        - GrLivArea (ground floor living area)
        - OverallQual (Overall quality of materials used)
        - TotalBsmtSF (The total of basement measured in sq. ft.)
        - YearBuilt (The year when the house was built)


DataCleaningEffect() taken from ML Feature Engine Unit 9: Custom Functions.

-  Function objective: assess the effect of the cleaning data when:
    - input mean, median aor arbitrary number is a numerical variable.
    - Replace with 'Missing' or most frequent a categorical variable.
- Parameters: 
    - df_original: data not being cleaned. 
    - df_cleaned: data being cleaned.
    - variables_applied_with_method: variables where i have applied a given method.


- It is understandable if, at first, the below code seems a little complicated, at this point we try to make sense of the pseud-code and understand the function parameters.  

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate the plot number
  
  # Distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over the variables, 
    # first on variables that you applied to the method
    # if the variable is numerical plot a histogram, if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1


___

## Data Cleaning

### Assessing the missing data levels from our dataset.

- We need to custom the function so this can show us the missing data levels in a data frame. 
    - Absolute levels.
    - Relative levels
    - Data type.

In [ ]:
def EvaluateMissingData(df):
  missing_data_absolute = df.isnull().sum()
  missing_data_percentage = round(missing_data_absolute/len(df)*100 , 2)
  df_missing_data = (pd.DataFrame(
                          data= {"RowsWithMissingData": missing_data_absolute,
                                 "PercentageOfDataset": missing_data_percentage,
                                 "DataType":df.dtypes}
                                  )
                    .sort_values(by=['PercentageOfDataset'],ascending=False)
                    .query("PercentageOfDataset > 0")
                    )

  return df_missing_data

- We have to evaluate the missing data:

In [ ]:
EvaluateMissingData(df)

- After the evaluation we can see the variables listed with the missing data.
- Therefore we will drop the Enclosed Porch and WoodDeckSF as there are more then 89.38, respectively 90.68 percent of null values.
- Other fields may possibly be imputed with a valid value or median.

*As we can see the 6 features that show positively and strongly correlation to the sale price of each house are not listed among these variables that contain null values.*

The 6 features that show positively correlation to the sale price are:
    - 1stFlrSF,
    - GarageArea,
    - GrLivArea,
    - OverallQuall,
    - TotalBsmtSF,
    - YearBuilt.

**At this point it is important to have a copy of the house price records data frame:**

In [ ]:
df_clean = df.copy()
print(df_clean.shape)

___

## Train and Test

- It is very important to split the dataset in 2 parts, so we will be able to test and train the ML.
    - It is also very important to split those in almost equal parts so we won't end up with not having enough data to train or test.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"train_set shape: {train_set.shape} \ntest_set shape: {test_set.shape}")

After we have split the dataset in the train set (1168, 24) and test_set (292, 24), we have to evaluate the missing values from train_set.

In [ ]:
df_missing_data = EvaluateMissingData(train_set)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

- After the evaluation from train_set, looks like there are 9 variables in total with missing data.
- Therefore, we will further investigate the 2 with a higher percentage of missing data.
    - EnclosedPorch (90.41%)
    - WoodDeckSF (88.53%)

In [ ]:
df_wooddecksf = train_set.loc[train_set['WoodDeckSF'].notnull()]
df_wooddecksf[['WoodDeckSF', 'SalePrice']]

In [ ]:
df_wooddecksf['WoodDeckSF'].value_counts().sort_index(ascending=False).head(10)

### EnclosedPorch - Enclosed porch area in square feet.

- During the evaluation of the missing data, we can clearly see that this variable contains more them 90% null values. (90.41%).
- According with the evaluation, we consider that this element won't be adding any value to the sale price.
- As the houses that my niece plans to buy do not have an enclosed porch I consider this element to be with no power of predictive sale price.

### WoodDeckSF - Wood deck area in square feet.

- During the evaluation of the missing data, we can clearly see that this variable contains more then 88% null values. (88.53%).
- According with the evaluation, we consider that this element won't be adding any value to the sale price.
- As the houses that my niece plans to buy do not add enough uniques to the houses, also the houses have to be refurbished before sale. 


Furthermore, I consider that the correlation and PPS analysis shows that these fields have no predictive power.

### Conclusion

Having in mind the above mentioned factors we consider that EnclosedPorch and WoodDeckSF add no value to the sale price, therefore we will be dropping this 2 factors.

- In order to drop these 2 factors we will be using the 'feature_engine's DropFeatures' method. 

In [ ]:
from feature_engine.selection import DropFeatures
variables = ['EnclosedPorch', 'WoodDeckSF']
imputer = DropFeatures(features_to_drop=variables)
imputer.fit(train_set)
train_set, test_set = imputer.transform(train_set), imputer.transform(test_set)
train_set.head()

We will clean the dataset for the refurbished houses as well.

In [ ]:
df_clean = imputer.transform(df_clean)

In [ ]:
null_variables = train_set.columns[train_set.isnull().any()].tolist()

___

## Variables to consider transforming or imputing.

- Inspect the LotFrontage and MasVnrArea variables.

- LotFrontage:

In [ ]:
train_set['LotFrontage'].value_counts().sort_index(ascending=False).head(10)

- MasVnrArea:

In [ ]:
train_set['MasVnrArea'].value_counts().sort_index(ascending=False).head(10)

- The PPS score on the above variables (LotFrontage and MasVnrArea), shows that these fields have no predictive power.
- The correlation study shows they have a moderate correlation to the sale price.
- On inspecting the dataset, for these variables, it is noted that in relation to other variables there is no way of identifying or deriving possible valid values for imputing on null variables. 

### In conclusion:

- Use MeanMedianImputer to impute a median value into the null variables:


In [ ]:
from feature_engine.imputation import MeanMedianImputer
variables = ['LotFrontage', 'MasVnrArea']
imputer = MeanMedianImputer(imputation_method='median', variables=variables)
imputer.fit(train_set)
train_set, test_set = imputer.transform(train_set), imputer.transform(test_set)

In [ ]:
df_clean = imputer.transform(df_clean)

In [ ]:
EvaluateMissingData(train_set)

- We can clearly see that after conducting our investigation and eliminating EnclosedPorch, WoodDeckSF and imputing null variables for LotFrontage and MasVnrArea they are not longer presented on our list.

- Further more, we will investigate the left elements to see their importance in the sale price. 

### 2ndFlrSf - Second floor square feet.

- Inspecting the 2ndFlrSF variable: 

In [ ]:
train_set['2ndFlrSF'].value_counts().sort_index()

- As stated above in the evaluating missing data, the 2nd floor square foot has 60 variables of 1168 which contain null variables.
- After a closer look at the dataset, it appears that if there is not a second floor, the value would be set to 0.
    - More then 50% of values for this variable is 0. Therefore, we deduce imputing the null values with 0 would add value to our dataset. 
- We prepare the pipeline to use ArbitraryNumberImputer to input 0 into the null variables. 

### BedroomAbvGr - Bedrooms above grade (dose NOT include basement bedrooms)

- Inspecting the BedroomAbvGr:

In [ ]:
train_set['BedroomAbvGr'].value_counts().sort_index()


- 80 variables of 1168 contains null values.
- Looking at the values we could see that there are only 4 records in our data set that contain 0 for bedrooms that are not included in basement. 
- As all the proprieties that my niece will refurbish contain values above 0, imputing the null values with 0 will not have an effect on the sales price analysis. 
- Furthermore, we will prepare the pipeline to use ArbitraryNumberImputer to input 0 into the null variables. 

In [ ]:
pipeline = Pipeline([( '2ndFlrSF',  ArbitraryNumberImputer(arbitrary_number=0, variables=['2ndFlrSF', 'BedroomAbvGr']))])
pipeline

In [ ]:
pipeline.fit(train_set)
train_set, test_set = pipeline.transform(train_set), pipeline.transform(test_set)

In [ ]:
df_clean = pipeline.transform(df_clean)

In [ ]:
EvaluateMissingData(train_set)

- After adjusting the pipeline we can see that 2ndFlrSF and BedroomAbvGr no longer appear on our list.

- Now we will investigate the BsmtFinType1.

### BsmtFinType1 - Rating of basement finished area.

- Inspecting BsmtFinType1 variable:

In [ ]:
train_set['BsmtFinType1'].value_counts().sort_index()

- Inspect BsmtExposure variable:


In [ ]:
train_set['BsmtExposure'].value_counts().sort_index()

In [ ]:

train_set[train_set['BsmtFinType1'].isna()].query('BsmtExposure=="None"').sort_values(by=['BsmtExposure'])


- 89 variables of 1168 contains null values.
- After inspecting the dataset, we will be able to see that there are only 25 properties with no basement.
- BsmtExposure however contains no null variables and on comparing the two fields i established that there are only 3 rows that are set to None.

- For these 3 rows the BsmtFinType1 variable can bee imputed with None.

In [ ]:
query_condition = (train_set.BsmtExposure == 'None') & (train_set['BsmtFinType1'].isnull())
train_set['BsmtFinType1'] = np.where(query_condition, 'None', train_set['BsmtFinType1'])

In [ ]:
query_condition = (test_set.BsmtExposure == 'None') & (test_set['BsmtFinType1'].isnull())
test_set['BsmtFinType1'] = np.where(query_condition, 'None', test_set['BsmtFinType1'])

In [ ]:
query_condition = (df_clean.BsmtExposure == 'None') & (df_clean['BsmtFinType1'].isnull())
df_clean['BsmtFinType1'] = np.where(query_condition, 'None', df_clean['BsmtFinType1'])

In [ ]:
train_set[train_set['BsmtFinType1'].isna()].query('BsmtExposure=="None"').sort_values(by=['BsmtExposure'])

- After the 3 rows have been imputed the value None, we are able to see that they are no longer appearing.

In [ ]:
train_set['BsmtFinType1'].isna().sum()

- We could see that there are still 86 BsmtFinType1 with a value of null.

- Furthermore, we will be inspecting BsmtFinSF1 variable. 

In [ ]:
df_temp = train_set[train_set['BsmtFinType1'].isna()].query('BsmtFinSF1==0').sort_values(by=['BsmtFinSF1'])
print(df_temp.shape)
df_temp

- Moving forward, we will be looking at BsmtFinSF1 which contains no null variables.
- I conducted a search and we could see that BsmtFinType1 has nulls and BsmtFinSF1 with value 0. In conclusion there are 0 finished squared feet, which means unfinished. 

- We have founded that there are BsmtFinType1 has 27 Unf, this means that 27 BsmtFinType1 is actually unfinished. 

In [ ]:
query_condition = (train_set.BsmtFinSF1 == 0) & (train_set['BsmtFinType1'].isnull())
train_set['BsmtFinType1'] = np.where(query_condition, 'Unf', train_set['BsmtFinType1'])

In [ ]:
query_condition = (test_set.BsmtFinSF1 == 0) & (test_set['BsmtFinType1'].isnull())
test_set['BsmtFinType1'] = np.where(query_condition, 'Unf', test_set['BsmtFinType1'])

In [ ]:
query_condition = (df_clean.BsmtFinSF1 == 0) & (df_clean['BsmtFinType1'].isnull())
df_clean['BsmtFinType1'] = np.where(query_condition, 'Unf', df_clean['BsmtFinType1'])

In [ ]:
train_set[train_set['BsmtFinType1'].isna()].query('BsmtFinSF1==0').sort_values(by=['BsmtFinSF1']).shape

- The 27 rows have been imputed with Unf, now those are not longer present in our dataset. 

In [ ]:
train_set['BsmtFinType1'].isna().sum()

- Looks like there are still 59 BsmtFinType1 containing a null variable.
- These remaining null variables will be imputed with Unk which means Unknown. 

In [ ]:
imputer = CategoricalImputer(imputation_method='missing',fill_value='Unk',
                             variables='BsmtFinType1')

imputer.fit(train_set)
train_set, test_set, df_clean = imputer.transform(train_set), imputer.transform(test_set), imputer.transform(df_clean)

In [ ]:
train_set['BsmtFinType1'].isna().sum()

- After imputing the Unk variables, there are no longer null values of BsmtFinType1.

### Garage Finish - Interior finish of the garage

- Inspecting the GarageFinish Variable.

In [ ]:
train_set['GarageFinish'].isna().sum()

- Looks like there are 131 variables that are null for GarageFinish variable. 

In [ ]:
train_set['GarageFinish'].value_counts().sort_index()

In [ ]:
train_set.loc[train_set.GarageFinish=="None",'GarageArea'].value_counts()

- After imputing GarageFinish=="None", meaning that the GarageFinish has been imputed with value None, the result is 0. There are no longer GarageArea.

In [ ]:
train_set.loc[train_set.GarageFinish.isnull(),'GarageArea'].value_counts()

- As the GarageFinish is null we are able to check if GarageArea is 0 and if so we can impute None on GarageFinish.
- Based on the above query, only 5 rows will be affected. 
- For the remaining records we will assume that the garages are unfinished so, we will impute Unf on Garage Finish.

In [ ]:
query_condition = (train_set.GarageArea == 0) & (train_set['GarageFinish'].isnull())
train_set['GarageFinish'] = np.where(query_condition, 'None', train_set['GarageFinish'])

In [ ]:
query_condition = (test_set.GarageArea == 0) & (test_set['GarageFinish'].isnull())
test_set['GarageFinish'] = np.where(query_condition, 'None', test_set['GarageFinish'])

In [ ]:
query_condition = (df_clean.GarageArea == 0) & (df_clean['GarageFinish'].isnull())
df_clean['GarageFinish'] = np.where(query_condition, 'None', df_clean['GarageFinish'])

In [ ]:
train_set['GarageFinish'].isna().sum()

In [ ]:
pipeline = Pipeline([
      ( 'categorical_imputer', CategoricalImputer(imputation_method='missing',
                                                  fill_value='Unf',
                                                  variables=['GarageFinish']) )
])
pipeline

In [ ]:
pipeline.fit(train_set)

train_set, test_set = pipeline.transform(train_set), pipeline.transform(test_set)

In [ ]:
df_clean = pipeline.transform(df_clean)

In [ ]:
train_set['GarageFinish'].isna().sum()

- After imputing the Unf on GarageFinish, we can see that there are no null values for GarageFinish. 

### GarageYrBlt - The year when the garage was built.

- Inspecting the GarageYrBlt variable.

In [ ]:
train_set['GarageYrBlt'].isna().sum()

- We have got the row count where GarageYrBlt is null, and return the value of GarageFinish.

In [ ]:
train_set.loc[train_set.GarageYrBlt.isnull(),'GarageFinish'].value_counts()

In [ ]:
train_set[train_set.GarageFinish=='None']

- We have to keep in mind that 58 null records are presented for the GarageYrBlt variable.
    - If the GarageYrBlt is null that means that the variable GarageFinish will be automatically none. This means that the property doesn't have a garage. 
- We will prepare the pipeline to use ArbitraryNumberImputer to impute 0 into the null variables:

In [ ]:
pipeline = Pipeline([
      ( 'GarageYrBlt',  ArbitraryNumberImputer(arbitrary_number=0,
                                                variables='GarageYrBlt') )
])
pipeline

pipeline.fit(train_set)
train_set, test_set = pipeline.transform(train_set), pipeline.transform(test_set)

In [ ]:
df_clean = pipeline.transform(df_clean)

- Now we have to see the missing data evaluation.

In [ ]:
EvaluateMissingData(train_set)

In [ ]:
EvaluateMissingData(df_clean)

- We can breath normally, as there are no longer variables missing data.

___

## Before and after comparison.

We now have to assess the effect on the variables distribution.

- The function plots in the same axes the distribution before and after applying the method.
- Now we will be able to see how different our variables would look after cleaning the data.
- We can notice a peak in the variable distribution after median imputation. 

In [ ]:
DataCleaningEffect(df_original=df,
                   df_cleaned=df_clean,
                   variables_applied_with_method=null_variables)

___

## Datatype changes - Float to integer.

- After examining tha data in the refurbished houses, we will be able to see that there are no float values in the float columns so we will change these into integers. 

In [ ]:
print(df_clean.shape)

In [ ]:
df_clean.select_dtypes('float').info()

In [ ]:
for col in df_clean.select_dtypes('float').columns:
    df_clean[col] = df_clean[col].astype('int64')

In [ ]:
df_clean.select_dtypes('float').info()

On examining the data in the refurbished houses, we can see that there are no float values. So we will change these into int.

In [ ]:
for col in df_refurbished.select_dtypes('float').columns:
    df_refurbished[col] = df_refurbished[col].astype('Int64')

In [ ]:
df_refurbished.select_dtypes('float').info()

In [ ]:
df_refurbished.info()

___

Change float columns to int for train and test.

In [ ]:
train_set.info()

In [ ]:
for col in train_set.select_dtypes('float').columns:
    train_set[col] = train_set[col].astype('int64')

In [ ]:
for col in test_set.select_dtypes('float').columns:
    test_set[col] = test_set[col].astype('int64')

___

## Save the training and test sets to csv.

- Create a cleaned folder:

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

- output the clean datasets to csv files into the outputs/datasets folder
- outputs/datasets/cleaned/train_set.csv
- outputs/datasets/cleaned/test_set.csv
- outputs/datasets/cleaned/clean_house_price_records.csv

In [ ]:
train_set.to_csv("outputs/datasets/cleaned/train_set.csv", index=False)

In [ ]:
test_set.to_csv("outputs/datasets/cleaned/test_set.csv", index=False)

In [ ]:
df_clean.to_csv("outputs/datasets/cleaned/clean_house_price_records.csv", index=False)

In [ ]:
df_refurbished.to_csv("outputs/datasets/cleaned/clean_refurbished_houses.csv", index=False)

___

## Conclusion and Next Steps

- We have created a cleaning version of the housing price dataset and the refurbished houses datasets.
- On the refurbished dataset the only step taken was to drop the variables EnclosedPorch and WoodDeckSf.
- The housing price were saved to csv files in the outputs/datasets/cleaned folder:
    - clean_house_price_records.csv
    - clean_refurbished_houses.csv
    - train_set.csv
    - test_set.csv

- Further, we will be moving on to Feature Engineering.

___